In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import time


#윈도우 만드는 함수 설정
def convert_to_timeseries(df, size):
    sequence_list = []
    
    for i in tqdm(range(0,df.shape[0] - size)):
        sequence_list.append(np.array(df.iloc[i:i+size,:]))
        
    sequence = np.array(sequence_list)
    
    return sequence


 # 데이터 불러오기   
df = pd.read_csv("/Users/kdlee/Downloads/vehicle/Out_1.csv", nrows =24242)


#x,y 라벨나눠주기
y_train = df.loc[:,['FLFX']]
x_train = df[df.columns.difference(['FLFX'])]

#스케일링 
#윈도우 생성
x_train=convert_to_timeseries(x_train,200)
y_train=convert_to_timeseries(y_train,200)



#신경망 구성
from tensorflow.keras.layers import LSTM, RNN, Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
    
step=200
n_inputs = 66
n_hiddens = 128
n_outputs = 1

inputs = Input(shape=(step, n_inputs))
hidden1 = LSTM(n_hiddens,return_sequences=True)(inputs)
hidden2 = LSTM(n_hiddens)(hidden1)
output = Dense(n_outputs,activation='relu')(hidden2)

model = Model(inputs,output)
model.summary()


#모델학습 얼리 스탑핑 넣어주면 넣어주고 안넣을려면 넣지 않도록 옵티마이저,loss 함수 등 알맞게 설정
early_stopping = EarlyStopping(patience = 5)
model.compile(optimizer='adam', loss='mae',metrics=['mae'])
history = model.fit(x_train, y_train, epochs= 50, batch_size=32,)


#모델 평가
scores = model.evaluate(x_train, y_train)
print("%s: %.2f%%" %(model.metrics_names[1], scores[1]*100))

#모델 예측
pred_out = model.predict(x_train< 여기에는 예측할 데이터)